<a href="https://colab.research.google.com/github/soichi-fujiwara/heroku-django-template/blob/master/wikiWord2VecCreGroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##学習済みDoc2Vecモデル
https://yag-ays.github.io/project/pretrained_doc2vec_wikipedia/

ライブラリ
Gensim: 3.6.0

In [103]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from gensim.models import KeyedVectors
from pprint import pprint

#Gドライブ保存後の確認
model_dir = './drive/My Drive/NLP/wiki_tohoku.model'
model = KeyedVectors.load(model_dir)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 2.8MB/s 


In [104]:
import pandas as pd
import itertools
#**************************************************************************
#分類特定
#**************************************************************************
is_a_parent = pd.read_table('./drive/My Drive/NLP/is_a_parent_list.txt')

#dataframeをlist化
is_a_parent_list = is_a_parent.values.tolist()
#2次元listから1次元listへ
is_a_parent_list = list(itertools.chain.from_iterable(is_a_parent_list))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [0]:
import pandas as pd

def group_check(child_wd):
  max_val = 0

  val_list = []
  val_list.append(child_wd)
  
  #is-a関係の判別
  for is_a_word in is_a_parent_list:
    try:
      parent_wd = is_a_word
      isa_val_list = []

      for i in range(1):
        rvs_wd = model.most_similar(positive=[child_wd])[i][0]
        if child_wd != rvs_wd.replace('[',"").replace(']',""):
          wk_val = model.similarity(parent_wd,rvs_wd)
          isa_val_list.append(wk_val)

          for ii in range(2):
            rvs_wd2 = model.most_similar(positive=[rvs_wd])[ii][0]
            if rvs_wd != rvs_wd2.replace('[',"").replace(']',""):
              wk_val = model.similarity(parent_wd,rvs_wd2)
              isa_val_list.append(wk_val)

    except KeyError as error:
      print(error)

    #辞書に登録の無い単語の場合は対応が必要
    max_val = max(isa_val_list)
    #print("【",parent_wd,"】is-a:",max_val)
    val_list.append(max_val)

  return val_list

In [0]:
import MeCab
cnt = 0

del_list =['(',')','-','"','、','。','/','.',',',':',')。',')、','「','」','『','』','()','』(','=','～','+','']

tokenizer = MeCab.Tagger("-Ochasen")
write_list = []

for k, v in model.vocab.items():

  if not k in del_list:
    node = tokenizer.parseToNode(k)
    while node:
      if node.feature.split(",")[0] == u"名詞":
        if(node.feature.split(",")[1] == u"サ変接続" or
           node.feature.split(",")[1] == u"一般" or
           node.feature.split(",")[1] == u"固有名詞"):
          #グループ判別
          rt_ave_val = group_check(k)
          #グループ判別値書き込み
          write_list.append(rt_ave_val)
          #print(write_list)
          cnt = cnt + 1
          print(cnt,"件目")
      else:
        pass
      node = node.next

      if cnt > 20:
        break
        

In [117]:
import pandas as pd

lst = is_a_parent_list.copy()
lst.insert(0,"WORD")

df = pd.DataFrame(data=write_list,columns=lst)
df2 = df.set_index('WORD')
df2

,男性,女性,キャラクター,職業,組織,施設,国家,都市,地域,自然,天体,道具,芸術,食べ物,乗り物,薬,本,ソフトウェア,動物,植物,身体,事件,武器,戦争,行事,天気,病気,電気,熱,火,色,歴史,経済,行為,言語,スポーツ,学問,現象,物,政治
WORD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
もの,0.165516,0.202869,0.313467,0.243459,0.249570,0.254798,0.282604,0.198885,0.266434,0.384550,0.346080,0.588025,0.164833,0.464533,0.478375,0.400181,0.268215,0.365054,0.497842,0.391358,0.401962,0.239652,0.423939,0.227626,0.235934,0.159459,0.167286,0.137966,0.352279,0.253935,0.365510,0.236584,0.143008,0.518972,0.403495,0.034106,0.355661,0.497961,1.000000,0.177245
中,0.114702,0.162599,0.119629,0.080010,0.045323,0.106696,0.129622,0.180225,0.189242,0.214600,0.152543,0.230351,0.141941,0.146658,0.198176,0.133931,0.153986,0.045166,0.216095,0.106006,0.176351,0.085490,0.183172,0.049685,0.120050,0.111164,0.115273,0.026155,0.157982,0.122202,0.096193,0.152146,0.134024,0.136713,0.112135,0.040466,0.174976,0.156587,0.170877,0.174001
者,0.410857,0.490812,0.210026,0.319572,0.194151,0.153386,0.369819,0.291739,0.347977,0.434771,0.300785,0.396952,0.297215,0.497099,0.352768,0.225435,0.053411,0.176058,0.463387,0.353130,0.348957,0.210382,0.244912,0.387542,0.317007,0.236987,0.310561,0.092155,0.256746,0.246236,0.111243,0.289246,0.284357,0.433058,0.380282,0.167726,0.432828,0.305042,0.379328,0.372398
時,0.200800,0.211979,0.141416,0.121480,0.062348,0.144244,0.053680,0.090928,0.239689,0.196432,0.219211,0.326548,-0.036326,0.233980,0.210659,0.202309,0.130513,0.079454,0.160754,0.162464,0.291919,0.207392,0.244641,0.197085,0.114309,0.097276,0.210028,0.106118,0.218817,0.177836,0.082521,0.009216,0.061311,0.349408,0.093509,0.004330,0.210587,0.295464,0.226500,0.075452
[日本],0.151768,0.207134,0.089553,0.219789,0.154002,0.264328,0.164434,0.086201,0.243584,0.089438,0.101899,0.085671,0.134736,0.074631,0.147401,0.115928,0.310719,0.290060,0.127176,0.106028,-0.006538,0.063795,0.071635,0.033497,0.089525,0.058569,-0.009168,0.146564,0.085370,-0.058081,0.094302,0.192932,0.095552,0.096753,0.120707,0.216800,0.086408,0.149988,0.108776,0.043302
[日本],0.151768,0.207134,0.089553,0.219789,0.154002,0.264328,0.164434,0.086201,0.243584,0.089438,0.101899,0.085671,0.134736,0.074631,0.147401,0.115928,0.310719,0.290060,0.127176,0.106028,-0.006538,0.063795,0.071635,0.033497,0.089525,0.058569,-0.009168,0.146564,0.085370,-0.058081,0.094302,0.192932,0.095552,0.096753,0.120707,0.216800,0.086408,0.149988,0.108776,0.043302
[日本],0.151768,0.207134,0.089553,0.219789,0.154002,0.264328,0.164434,0.086201,0.243584,0.089438,0.101899,0.085671,0.134736,0.074631,0.147401,0.115928,0.310719,0.290060,0.127176,0.106028,-0.006538,0.063795,0.071635,0.033497,0.089525,0.058569,-0.009168,0.146564,0.085370,-0.058081,0.094302,0.192932,0.095552,0.096753,0.120707,0.216800,0.086408,0.149988,0.108776,0.043302
名,0.229963,0.242619,0.311333,0.287530,0.168273,0.045140,0.079204,0.107826,0.252945,0.127213,0.198091,0.288554,0.050697,0.315537,0.286764,0.147898,0.153102,0.121289,0.177036,0.184035,0.065964,0.165336,0.124604,0.105027,0.309115,0.139707,0.171942,0.040894,0.107827,0.178530,0.323514,0.242417,-0.072919,0.216001,0.297920,0.119267,0.193930,0.359741,0.211321,0.022964
ば,0.105123,0.127423,0.099894,0.196893,0.060786,0.094751,0.155137,0.083398,0.084447,0.246963,0.140406,0.247045,0.071882,0.220413,0.194082,0.173763,-0.000497,0.113375,0.194512,0.127835,0.289976,0.113513,0.221801,0.194795,0.018009,0.218795,0.140305,0.123845,0.206254,0.217736,0.107773,-0.019027,0.126789,0.300194,0.195293,0.067739,0.200406,0.254449,0.251428,0.141722


In [121]:
model.most_similar(positive=[u"神様"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('王様', 0.6576182842254639),
 ('天国', 0.6466959714889526),
 ('お母さん', 0.6404556632041931),
 ('おやじ', 0.6315042972564697),
 ('父さん', 0.6275973320007324),
 ('小僧', 0.6270861625671387),
 ('お父さん', 0.6250303983688354),
 ('お守り', 0.624343752861023),
 ('おじいさん', 0.6236745119094849),
 ('巫女', 0.6210343241691589)]

In [94]:
model.similarity("10月","時代")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.20133215